<a href="https://colab.research.google.com/github/Ditipriya5678/Coronavirus-genome-sequence/blob/main/Covid_model_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from Bio import SeqIO
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
records = SeqIO.parse("coronavirus.fasta", "fasta")
sequences = [str(record.seq) for record in records]

# Map nucleotides to integers
nucleotide_dict = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
X = [[nucleotide_dict[nucleotide] for nucleotide in sequence] for sequence in sequences]

# Pad the sequences so that they all have the same length
X = pad_sequences(X)

# Create the target labels
y = np.zeros(len(X))
y[:int(len(X)/2)] = 1
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# Define the model architecture
model = Sequential()
model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
from keras.optimizers import Adam

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


# Compile the model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

<ipython-input-4-1cfa3556e38c>:28: RuntimeWarning: invalid value encountered in true_divide
  X_train = (X_train - mean) / std
<ipython-input-4-1cfa3556e38c>:29: RuntimeWarning: divide by zero encountered in true_divide
  X_test = (X_test - mean) / std
<ipython-input-4-1cfa3556e38c>:29: RuntimeWarning: invalid value encountered in true_divide
  X_test = (X_test - mean) / std
/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10


/usr/local/lib/python3.9/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 2s 2s/step - loss: nan - accuracy: 0.5714 - val_loss: nan - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 84ms/step - loss: nan - accuracy: 0.5714 - val_loss: nan - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 62ms/step - loss: nan - accuracy: 0.5714 - val_loss: nan - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 79ms/step - loss: nan - accuracy: 0.5714 - val_loss: nan - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 74ms/step - loss: nan - accuracy: 0.5714 - val_loss: nan - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 62ms/step - loss: nan - accuracy: 0.5714 - val_loss: nan - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 93ms/step - loss: nan - accuracy: 0.5714 - val_loss: nan - val_accuracy: 0.5000
Epoch 8/10
1/1 [==============================] - 0s 77ms/step - loss: nan - accur

In [ ]:
import numpy as np
from Bio import SeqIO
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
records = SeqIO.parse("coronavirus.fasta", "fasta")
sequences = [str(record.seq) for record in records]

# Define the length of the k-mer
k = 4

# Create a dictionary to map k-mers to integers
kmer_dict = {}
kmer_idx = 0
for i in range(4**k):
    kmer = ""
    for j in range(k):
        kmer += "ACGT"[int(i / (4**(k-j-1))) % 4]
    kmer_dict[kmer] = kmer_idx
    kmer_idx += 1

# Convert the sequences to k-mer counts
X = []
for sequence in sequences:
    counts = np.zeros(len(kmer_dict))
    for i in range(len(sequence)-k+1):
        kmer = sequence[i:i+k]
        if kmer in kmer_dict:
            counts[kmer_dict[kmer]] += 1
    X.append(counts)
X = np.array(X)

# Create the target labels
y = np.zeros(len(X))
y[:int(len(X)/2)] = 1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the features
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# Define the model architecture
model = Sequential()
model.add(Dense(50, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='relu'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)


Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 7.7561 - accuracy: 0.5000 - val_loss: 5.1416 - val_accuracy: 0.6667
Epoch 2/10
1/1 [==============================] - 0s 45ms/step - loss: 2.9517 - accuracy: 0.5000 - val_loss: 5.1416 - val_accuracy: 0.6667
Epoch 3/10
1/1 [==============================] - 0s 38ms/step - loss: 5.7024 - accuracy: 0.3333 - val_loss: 5.1416 - val_accuracy: 0.6667
Epoch 4/10
1/1 [==============================] - 0s 37ms/step - loss: 8.1378 - accuracy: 0.1667 - val_loss: 5.1416 - val_accuracy: 0.6667
Epoch 5/10
1/1 [==============================] - 0s 35ms/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 5.1416 - val_accuracy: 0.6667
Epoch 6/10
1/1 [==============================] - 0s 34ms/step - loss: 7.7202 - accuracy: 0.5000 - val_loss: 5.1416 - val_accuracy: 0.6667
Epoch 7/10
1/1 [==============================] - 0s 35ms/step - loss: 7.7471 - accuracy: 0.5000 - val_loss: 5.1416 - val_accuracy: 0.6667
Epoch 8/10
1/1 [=============